In [8]:
# import sys
# !{sys.executable} -m pip install billboard.py
# #!{sys.executable} -m pip install pandas


In [9]:
import billboard as bb
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta as td

In [10]:
def BillBoard_Hot100_Parser(numSongsToTrack=3, numYears=1, isVerbose=False):
    """
    FUNCTIONALITY:
        Takes chart to follow, number of top songs to track, number of years to go back, and a verbosity flag.
        Prints progress during execution and prints top 5 rows upon compeletion if verbosity flag is set to true.
        Pulls the billboard hot 100 chart for the number of years back from today (current week).
        Adds the numSongsToTrack top songs to pandas dataframe w/ 1 Row = 1 Week.
        Dataframe columns = beginDate (for the week of the chart), endDate, artist<i>, title<i>,
        peakPosition<i>, positionLastWeek<i>,  weeksOnChart<i>. <i> is the ranking of the current song.
        For example the #1 ranked song of the week is artist1, title1, etc.
        There is one artist column, one title column... etc for each song tracked as passed in by the numSongsToTrack parameter.
    RETURN VALUE:
        Pandas DataFrame
    ERRORS:
        if numSongsToTrack <= 0 or > 100 Prints Error Message and returns None.
        If numYears <= 0 Prints Error Message and returns None.
        If numYears exceeds total history of billboard charts, prints warning and returns dataframe up to end of charts.
    NOTES:
        Takes a while; be patient and use the verbosity flag if you want to make sure it is working.
        How long it takes is internet-speed dependent."""
    
    
    if numSongsToTrack <= 0 or numSongsToTrack > 100:
        print("Error: Num Songs cannont less than or equal to 0 or greater than 100.")
        return None
    if numYears <= 0:
        print("Error: Num Years cannont be less than or equal to 0.")
        return None
    
    
    df = pd.DataFrame()
    
    #Gets current chart
    chart = bb.ChartData('hot-100')

    numWeeksGotten = 0
    numWeeks = numYears * 52
    if isVerbose:
        print("Total Number of Weeks To Process: {}\n".format(numWeeks))
    
    #while there is another chart to get and number of weeks doesn't exceed that passed in.
    while (chart.date and numWeeksGotten < numWeeks):
        
         #catches error where billbard interface doesn't return a chart, just gets chart for -1 day from date.
        try:
            testVar = chart[0].title
        except:
            dateTimeStart = dt.strptime(endDate, "%Y-%m-%d")
            dateTimeStartMinusOne = dateTimeStart + td(days=(-1))
            strBeginDate = dateTimeStartMinusOne.strftime("%Y-%m-%d")
            chart = bb.ChartData('hot-100', strBeginDate)
        
        #converts adds one to chart previous date through python's datetime module
        #to get correct begining date for chart
        endDate = chart.date
        previousDate = chart.previousDate
        dateTimePrevious = dt.strptime(previousDate, "%Y-%m-%d")
        dateTimePreviousPlusOne = dateTimePrevious + td(days=1)
        strBeginDate = dateTimePreviousPlusOne.strftime("%Y-%m-%d")
        
        #appends info to a temporary dictionary that later is converted to a dataframe
        dfToAppend = {}
        dfToAppend['endDate'] = endDate
        dfToAppend['beginDate'] = strBeginDate
        
        #iterates through the correct numSongs as passed above and adds to dataframe.
        for ii in range(0, numSongsToTrack):

            strII = str(ii+1)

            strTitle = 'title_'+strII
            dfToAppend[strTitle] = chart[ii].title

            strArtist = 'artist_' + strII
            dfToAppend[strArtist] = chart[ii].artist

            strPeakPos = 'peakPosition_' + strII
            dfToAppend[strPeakPos] = chart[ii].peakPos

            strLastPos = 'positionLastWeek_' + strII
            dfToAppend[strLastPos] = chart[ii].lastPos

            strWeeksOnChart = 'weeksOnChart_' + strII
            dfToAppend[strWeeksOnChart] = chart[ii].weeks
        
         #converts dict to dataframe   
        dfToAppend =pd.DataFrame(dfToAppend, index =[endDate])
        
        #appends dataframe to whole dataframe
        df = df.append(dfToAppend)
        
        numWeeksGotten += 1
        if isVerbose:
            print("Number of Weeks Processed: {}\n".format(numWeeksGotten))
            
        dfToAppend = None
        
        #gets previous chart
        chart = bb.ChartData('hot-100', chart.previousDate)
        
           
    if numWeeksGotten < numWeeks:
        print("Warning: numYears exceeds total history of top-100 chart.")
        
    if isVerbose:
        print(df.head())
            
    return df

In [11]:
def get_All_Time_200(numToGet=200):
    """
    FUNCTIONALITY:
        Takes number of songs to get and retrieves the top <numToGet> songs from the billboard all-time top 200 Chart.
        Creates Pandas DataFrame with artist, song title, rank.
    RETURN VALUE:
        Pandas DataFrame
    Errors:
        Prints error message and returns None if numToGet <= 0.
        Prints warning and returns full chart if numToGet >= 200"""
    
    #Error Messages
    if numToGet <= 0:
        print("Error: Cannot Get 0 or Less Songs")
        return None
    if numToGet > 200:
        print("Warning: Cannont get more than 200 Songs. Full Chart Returned")
        
     #creating dataframe and getting chart   
    df = pd.DataFrame()
    chart = bb.ChartData('greatest-billboard-200-albums')

    #appends info about one song to a dictionary, then converts to data frame, then appends to original dataframe.
    for ii in range(numToGet):

            dictToMakeDF = {}

            dictToMakeDF['title'] = chart[ii].title

            dictToMakeDF['artist'] = chart[ii].artist
            
            dictToMakeDF['rank'] = chart[ii].rank

            dfToAppend = pd.DataFrame(dictToMakeDF, index=[ii+1])

            df = df.append(dfToAppend)
            
            
    return df

In [12]:
df = BillBoard_Hot100_Parser(numSongsToTrack=100, numYears=53, isVerbose=True)
df = get_All_Time_200(numToGet=200)

Total Number of Weeks To Process: 2756

Number of Weeks Processed: 1

Number of Weeks Processed: 2

Number of Weeks Processed: 3

Number of Weeks Processed: 4

Number of Weeks Processed: 5

Number of Weeks Processed: 6

Number of Weeks Processed: 7

Number of Weeks Processed: 8

Number of Weeks Processed: 9

Number of Weeks Processed: 10

Number of Weeks Processed: 11

Number of Weeks Processed: 12

Number of Weeks Processed: 13

Number of Weeks Processed: 14

Number of Weeks Processed: 15

Number of Weeks Processed: 16

Number of Weeks Processed: 17

Number of Weeks Processed: 18

Number of Weeks Processed: 19

Number of Weeks Processed: 20

Number of Weeks Processed: 21

Number of Weeks Processed: 22

Number of Weeks Processed: 23

Number of Weeks Processed: 24

Number of Weeks Processed: 25

Number of Weeks Processed: 26

Number of Weeks Processed: 27

Number of Weeks Processed: 28

Number of Weeks Processed: 29

Number of Weeks Processed: 30

Number of Weeks Processed: 31

Number o

Number of Weeks Processed: 260

Number of Weeks Processed: 261

Number of Weeks Processed: 262

Number of Weeks Processed: 263

Number of Weeks Processed: 264

Number of Weeks Processed: 265

Number of Weeks Processed: 266

Number of Weeks Processed: 267

Number of Weeks Processed: 268

Number of Weeks Processed: 269

Number of Weeks Processed: 270

Number of Weeks Processed: 271

Number of Weeks Processed: 272

Number of Weeks Processed: 273

Number of Weeks Processed: 274

Number of Weeks Processed: 275

Number of Weeks Processed: 276

Number of Weeks Processed: 277

Number of Weeks Processed: 278

Number of Weeks Processed: 279

Number of Weeks Processed: 280

Number of Weeks Processed: 281

Number of Weeks Processed: 282

Number of Weeks Processed: 283

Number of Weeks Processed: 284

Number of Weeks Processed: 285

Number of Weeks Processed: 286

Number of Weeks Processed: 287

Number of Weeks Processed: 288

Number of Weeks Processed: 289

Number of Weeks Processed: 290

Number o

Number of Weeks Processed: 517

Number of Weeks Processed: 518

Number of Weeks Processed: 519

Number of Weeks Processed: 520

Number of Weeks Processed: 521

Number of Weeks Processed: 522

Number of Weeks Processed: 523

Number of Weeks Processed: 524

Number of Weeks Processed: 525

Number of Weeks Processed: 526

Number of Weeks Processed: 527

Number of Weeks Processed: 528

Number of Weeks Processed: 529

Number of Weeks Processed: 530

Number of Weeks Processed: 531

Number of Weeks Processed: 532

Number of Weeks Processed: 533

Number of Weeks Processed: 534

Number of Weeks Processed: 535

Number of Weeks Processed: 536

Number of Weeks Processed: 537

Number of Weeks Processed: 538

Number of Weeks Processed: 539

Number of Weeks Processed: 540

Number of Weeks Processed: 541

Number of Weeks Processed: 542

Number of Weeks Processed: 543

Number of Weeks Processed: 544

Number of Weeks Processed: 545

Number of Weeks Processed: 546

Number of Weeks Processed: 547

Number o

Number of Weeks Processed: 774

Number of Weeks Processed: 775

Number of Weeks Processed: 776

Number of Weeks Processed: 777

Number of Weeks Processed: 778

Number of Weeks Processed: 779

Number of Weeks Processed: 780

Number of Weeks Processed: 781

Number of Weeks Processed: 782

Number of Weeks Processed: 783

Number of Weeks Processed: 784

Number of Weeks Processed: 785

Number of Weeks Processed: 786

Number of Weeks Processed: 787

Number of Weeks Processed: 788

Number of Weeks Processed: 789

Number of Weeks Processed: 790

Number of Weeks Processed: 791

Number of Weeks Processed: 792

Number of Weeks Processed: 793

Number of Weeks Processed: 794

Number of Weeks Processed: 795

Number of Weeks Processed: 796

Number of Weeks Processed: 797

Number of Weeks Processed: 798

Number of Weeks Processed: 799

Number of Weeks Processed: 800

Number of Weeks Processed: 801

Number of Weeks Processed: 802

Number of Weeks Processed: 803

Number of Weeks Processed: 804

Number o

Number of Weeks Processed: 1030

Number of Weeks Processed: 1031

Number of Weeks Processed: 1032

Number of Weeks Processed: 1033

Number of Weeks Processed: 1034

Number of Weeks Processed: 1035

Number of Weeks Processed: 1036

Number of Weeks Processed: 1037

Number of Weeks Processed: 1038

Number of Weeks Processed: 1039

Number of Weeks Processed: 1040

Number of Weeks Processed: 1041

Number of Weeks Processed: 1042

Number of Weeks Processed: 1043

Number of Weeks Processed: 1044

Number of Weeks Processed: 1045

Number of Weeks Processed: 1046

Number of Weeks Processed: 1047

Number of Weeks Processed: 1048

Number of Weeks Processed: 1049

Number of Weeks Processed: 1050

Number of Weeks Processed: 1051

Number of Weeks Processed: 1052

Number of Weeks Processed: 1053

Number of Weeks Processed: 1054

Number of Weeks Processed: 1055

Number of Weeks Processed: 1056

Number of Weeks Processed: 1057

Number of Weeks Processed: 1058

Number of Weeks Processed: 1059

Number of 

...

Number of Weeks Processed: 1528

Number of Weeks Processed: 1529

Number of Weeks Processed: 1530

Number of Weeks Processed: 1531

Number of Weeks Processed: 1532

Number of Weeks Processed: 1533

Number of Weeks Processed: 1534

Number of Weeks Processed: 1535

Number of Weeks Processed: 1536

Number of Weeks Processed: 1537

Number of Weeks Processed: 1538

Number of Weeks Processed: 1539

Number of Weeks Processed: 1540

Number of Weeks Processed: 1541

Number of Weeks Processed: 1542

Number of Weeks Processed: 1543

Number of Weeks Processed: 1544

Number of Weeks Processed: 1545

Number of Weeks Processed: 1546

Number of Weeks Processed: 1547

Number of Weeks Processed: 1548

Number of Weeks Processed: 1549

Number of Weeks Processed: 1550

Number of Weeks Processed: 1551

Number of Weeks Processed: 1552

Number of Weeks Processed: 1553

Number of Weeks Processed: 1554

Number of Weeks Processed: 1555

Number of Weeks Processed: 1556

Number of Weeks Processed: 1557

Number of 

Number of Weeks Processed: 1777

Number of Weeks Processed: 1778

Number of Weeks Processed: 1779

Number of Weeks Processed: 1780

Number of Weeks Processed: 1781

Number of Weeks Processed: 1782

Number of Weeks Processed: 1783

Number of Weeks Processed: 1784

Number of Weeks Processed: 1785

Number of Weeks Processed: 1786

Number of Weeks Processed: 1787

Number of Weeks Processed: 1788

Number of Weeks Processed: 1789

Number of Weeks Processed: 1790

Number of Weeks Processed: 1791

Number of Weeks Processed: 1792

Number of Weeks Processed: 1793

Number of Weeks Processed: 1794

Number of Weeks Processed: 1795

Number of Weeks Processed: 1796

Number of Weeks Processed: 1797

Number of Weeks Processed: 1798

Number of Weeks Processed: 1799

Number of Weeks Processed: 1800

Number of Weeks Processed: 1801

Number of Weeks Processed: 1802

Number of Weeks Processed: 1803

Number of Weeks Processed: 1804

Number of Weeks Processed: 1805

Number of Weeks Processed: 1806

Number of 

Number of Weeks Processed: 2026

Number of Weeks Processed: 2027

Number of Weeks Processed: 2028

Number of Weeks Processed: 2029

Number of Weeks Processed: 2030

Number of Weeks Processed: 2031

Number of Weeks Processed: 2032

Number of Weeks Processed: 2033

Number of Weeks Processed: 2034

Number of Weeks Processed: 2035

Number of Weeks Processed: 2036

Number of Weeks Processed: 2037

Number of Weeks Processed: 2038

Number of Weeks Processed: 2039

Number of Weeks Processed: 2040

Number of Weeks Processed: 2041

Number of Weeks Processed: 2042

Number of Weeks Processed: 2043

Number of Weeks Processed: 2044

Number of Weeks Processed: 2045

Number of Weeks Processed: 2046

Number of Weeks Processed: 2047

Number of Weeks Processed: 2048

Number of Weeks Processed: 2049

Number of Weeks Processed: 2050

Number of Weeks Processed: 2051

Number of Weeks Processed: 2052

Number of Weeks Processed: 2053

Number of Weeks Processed: 2054

Number of Weeks Processed: 2055

Number of 

Number of Weeks Processed: 2275

Number of Weeks Processed: 2276

Number of Weeks Processed: 2277

Number of Weeks Processed: 2278

Number of Weeks Processed: 2279

Number of Weeks Processed: 2280

Number of Weeks Processed: 2281

Number of Weeks Processed: 2282

Number of Weeks Processed: 2283

Number of Weeks Processed: 2284

Number of Weeks Processed: 2285

Number of Weeks Processed: 2286

Number of Weeks Processed: 2287

Number of Weeks Processed: 2288

Number of Weeks Processed: 2289

Number of Weeks Processed: 2290

Number of Weeks Processed: 2291

Number of Weeks Processed: 2292

Number of Weeks Processed: 2293

Number of Weeks Processed: 2294

Number of Weeks Processed: 2295

Number of Weeks Processed: 2296

Number of Weeks Processed: 2297

Number of Weeks Processed: 2298

Number of Weeks Processed: 2299

Number of Weeks Processed: 2300

Number of Weeks Processed: 2301

Number of Weeks Processed: 2302

Number of Weeks Processed: 2303

Number of Weeks Processed: 2304

Number of 

Number of Weeks Processed: 2524

Number of Weeks Processed: 2525

Number of Weeks Processed: 2526

Number of Weeks Processed: 2527

Number of Weeks Processed: 2528

Number of Weeks Processed: 2529

Number of Weeks Processed: 2530

Number of Weeks Processed: 2531

Number of Weeks Processed: 2532

Number of Weeks Processed: 2533

Number of Weeks Processed: 2534

Number of Weeks Processed: 2535

Number of Weeks Processed: 2536

Number of Weeks Processed: 2537

Number of Weeks Processed: 2538

Number of Weeks Processed: 2539

Number of Weeks Processed: 2540

Number of Weeks Processed: 2541

Number of Weeks Processed: 2542

Number of Weeks Processed: 2543

Number of Weeks Processed: 2544

Number of Weeks Processed: 2545

Number of Weeks Processed: 2546

Number of Weeks Processed: 2547

Number of Weeks Processed: 2548

Number of Weeks Processed: 2549

Number of Weeks Processed: 2550

Number of Weeks Processed: 2551

Number of Weeks Processed: 2552

Number of Weeks Processed: 2553

Number of 

In [13]:
df.to_json(path_or_buf="hot_100_charts.json", orient='index')